<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Complete_DARTS_Cell_Implementation_with_Architecture_Search_Logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# --- DARTS-inspired Cell ---
class DARTSCell(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DARTSCell, self).__init__()
        self.op1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.op2 = nn.Conv2d(in_channels, out_channels, kernel_size=5, padding=2)

        # Learnable architecture parameter (initialized near 0.5 for a fair start)
        self.alpha = nn.Parameter(torch.tensor(0.5))

    def forward(self, x):
        # Optional: constrain alpha to [0, 1] using sigmoid
        alpha_clipped = torch.sigmoid(self.alpha)
        out1 = self.op1(x)
        out2 = self.op2(x)
        return alpha_clipped * out1 + (1 - alpha_clipped) * out2

# --- Example Training Step ---
if __name__ == "__main__":
    torch.manual_seed(42)

    model = DARTSCell(in_channels=3, out_channels=16)
    optimizer = optim.Adam([
        {'params': model.op1.parameters()},
        {'params': model.op2.parameters()},
        {'params': [model.alpha], 'lr': 0.01}  # Separate learning rate for alpha if needed
    ], lr=0.001)

    x = torch.randn(4, 3, 32, 32)  # Batch of 4 RGB images (32x32)

    for step in range(1, 6):
        optimizer.zero_grad()
        output = model(x)
        loss = output.mean()  # Dummy loss for gradient flow demo
        loss.backward()
        optimizer.step()

        print(f"Step {step} | Loss: {loss.item():.4f} | α (raw): {model.alpha.item():.4f} | α (sigmoid): {torch.sigmoid(model.alpha).item():.4f}")